In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import glob
import pickle
import numpy as np

In [3]:
!pip install music21

In [4]:
from music21 import converter, instrument, note, chord

In [5]:
def train_network():
  notes = get_notes()
  n_vocab = len(set(notes))
  network_input, network_output = prepare_sequences(notes, n_vocab)

  model = create_netowork(network_input, n_vocab)
  train(model, network_input, network_output)

In [6]:
def get_notes():
  notes = []

  for file in glob.glob('/content/drive/MyDrive/Projects/Music Generation using LSTM/Classical-Piano-Composer-master/midi_songs/*.mid'):
    midi = converter.parse(file)
    print('Parsing is', file)
    notes_to_parse = None

    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi)
      notes_to_parse = s2.parts[0].recurse()
    
    except: # file does not have instrument parts
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/drive/MyDrive/Projects/Music Generation using LSTM/Classical-Piano-Composer-master/data/notes', 'wb') as filepath:
      pickle.dump(notes, filepath)
    
    return notes

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [8]:
def prepare_sequences(notes, n_vocab):
  sequence_length = 100

  # get all pitvh names
  pitchnames = sorted(set(item for item in notes))

  # create a dictionary to match pitches to integers
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []

  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

  # reshape the input into a format compatible with LSTM Layers
  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
  # normalize input
  network_input = network_input / float(n_vocab)

  network_output = np_utils.to_categorical(network_output)
  return (network_input, network_output)

In [9]:
def create_netowork(network_input, n_vocab):
  model = Sequential()
  model.add(LSTM(
      512,
      input_shape = (network_input.shape[1], network_input.shape[2]),
      recurrent_dropout = 0.3,
      return_sequences = True
  ))
  model.add(LSTM(512, return_sequences = True, recurrent_dropout = 0.3))
  model.add(LSTM(512))
  model.add(BatchNorm())
  model.add(Dropout(0.3))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(BatchNorm())
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')

  return model

In [10]:
def train(model, network_input, network_output):
  filepath = '/content/drive/MyDrive/Projects/Music Generation using LSTM/Classical-Piano-Composer-master/new_weights.hdf5'
  checkpoint = ModelCheckpoint(
      filepath, 
      monitor = 'loss',
      verbose = 0,
      save_best_only = True,
      mode = 'min'
  )
  
  callbacks_list = [checkpoint]
  model.fit(network_input, network_output, epochs = 200, batch_size = 120, callbacks = callbacks_list)

In [11]:
if __name__ == '__main__':
  train_network()

Parsing is /content/drive/MyDrive/Projects/Music Generation using LSTM/Classical-Piano-Composer-master/midi_songs/FFVII_BATTLE.mid
Epoch 1/200
3/3 [==============================] - 16s 1s/step - loss: 4.6034
Epoch 2/200
3/3 [==============================] - 4s 1s/step - loss: 4.6050
Epoch 3/200
3/3 [==============================] - 4s 1s/step - loss: 4.5132
Epoch 4/200
3/3 [==============================] - 4s 1s/step - loss: 4.7379
Epoch 5/200
3/3 [==============================] - 4s 1s/step - loss: 4.4539
Epoch 6/200
3/3 [==============================] - 4s 1s/step - loss: 4.4620
Epoch 7/200
3/3 [==============================] - 4s 1s/step - loss: 4.4274
Epoch 8/200
3/3 [==============================] - 4s 1s/step - loss: 4.4570
Epoch 9/200
3/3 [==============================] - 4s 1s/step - loss: 4.0385
Epoch 10/200
3/3 [==============================] - 4s 1s/step - loss: 4.2452
Epoch 11/200
3/3 [==============================] - 4s 1s/step - loss: 4.2124
Epoch 12/200
3/3 [=